In [ ]:
# Lectura del archivo del grafo
def init():
    archivo = sc.textFile('gc_500_2')
    tam = archivo.take(1)[0]
    aristas = archivo.filter(lambda l: l != tam)
    
    tam2 = tam.split(' ') #tam[0]: |V|  tam[1]: |E|
    vertices = range(0, int(tam2[0]))

    aristas = aristas.take(int(tam2[1]))
    
    # Creo el grafo a partir de dos RDDs: uno de vértices y otro de aristas
    # Cada vértice es  una tupla (numeroVertice, colorVertice) con color inicializado en 0.
    grafo_V = sc.parallelize(vertices, 8).map(lambda v: (v, 0))
    # Cada arista es una tupla (u, v) donde "u" y "v" son vértices del grafo
    grafo_E = sc.parallelize(aristas).map(lambda e: e.split(' ')).map(lambda e: (int(e[0]), int(e[1])))
    #print grafo_E.take(10)
    #grafo_E.collect()
    return (grafo_V, grafo_E)
                          

In [ ]:
# Algunas funciones útiles:

# Devuelve true si u y v están conectados o false en caso contrario
def estanConectados(u, v):
    aux = grafo_E.filter(lambda e: (e[1]== u and e[0]== v) or (e[0]== u and e[1]== v))
    return (len(aux.take(1)) > 0)

# Devuelve true si u y v tienen el mismo color en el grafo
def verticesConMismoColor(u, v):
    aux = grafo_V.filter(lambda x: (x[0] == v) or (x[0] == u)).take(2)
    return (aux[1][1] == aux[0][1])

#Devuelve un RDD con los numeros de vertices adyacentes a v
def verticesAdyacentes(v):
    aux = grafo_E.filter(lambda e: (e[0] == v) or (e[1] == v))
    return aux.map(lambda x: x[1] if(x[0] == v) else x[0])
    
    
# Devuelve true si el vértice v tiene un coloreo válido.
# El coloreo de v es válido sii v tiene color 0 o bien v tiene un color
# distinto al de todos sus vértices vecinos
def colorVerticeValido(v):
    #colorV = grafo_V.filter(lambda x: x[0] == v).first()[1]
    #if (colorV == 0):
    #    return True
    #aux = grafo_E.filter(lambda e: (e[0] == v) or (e[1] == v)).map(lambda x: (x[1], 1) if(x[0] == v) else (x[0], 1))
    #aux = grafo_V.subtractByKey(aux)
    #aux = grafo_V.subtractByKey(aux)
    # Tengo guardado en aux los pares (vAdyacente, color)
    #return aux.map(lambda x: x[1] != colorV).reduce(lambda a, b: a and b)
    color = grafo_V.filter(lambda x: x[0] == v).first()[1]
    if (color == 0):
        return True
    # Busco todos los adyacentes a v y los que tienen su mismo color
    try:
        ady = verticesAdyacentes(v)
        aux = grafo_V.filter(lambda x: (x[1] == color) and (x[0] != v)).map(lambda x: x[0])
        aux = aux.intersection(ady)
        #print 'Ponele: ', aux.collect()
        return (len(aux.take(1)) == 0)
    except:
        return False
    
# Devuelve true si el coloreo actual del grafo es válido o false si no lo es
# Un coloreo no es válido sii dos vértices adyacentes tienen el mismo color y
# este es distinto de 0 (Ergo, esta función devuelve true si hay vértices
# adyacentes con color 0)
def coloreoEsValido():
    cantV = grafo_V.count()
    for x in range (0, cantV):
        if (colorVerticeValido(x) == False):
            return False
    return True

sc.setCheckpointDir('checkpoint')

In [ ]:
# Forma de coloreo #1: "Fuerza bruta"
# Asigna un color random en el rango (1, colorMax)
import random
def colorearRandom():
    return random.randint(1, colores)


import os
import numpy as np

def foo(v):
    return losColores[v]
    
# Parte del programa encargada de asignar un color a los vértices
(grafo_V, grafo_E) = init()

termine = False
colores = 26
iterMax = 800
i = 0
tope = grafo_V.count()
intentosPorColor = 30

while((termine == False) and (i < iterMax)):
    # Generador random
    losColores = []
    for x in range (0, tope):
        losColores.append(random.randint(1, colores))

    grafo_V = grafo_V.map(lambda v: (v[0], foo(v[0])))
    if(not coloreoEsValido()):
        #print 'Coloreo invalido: ', grafo_V.collect()
        termine = False
    else:
        #print 'vxffdColoreo caliaddo: ', grafo_V.collect()
        termine = True
    i += 1
    if((i % intentosPorColor) == 0):
        colores += 1
        print 'Aumento colores al valor: ', colores
# Mostrar resultados      
print '-----Resultados-----'
print 'Coloreo final: ', grafo_V.collect()
print 'Iteraciones consumidas: ', i
coloresUsados = grafo_V.map(lambda x: (x[1], x[0])).groupByKey().count()
print 'Cantidad de colores distintos usados: ', coloresUsados

In [ ]:
# Forma de coloreo #2: "Greedy según Welsh-Powell"
# Se ordenan los nodos de mayor a menor grado (cant. de aristas incidentes)
# Se intenta colorear primero los nodos de MAYOR grado con el menor color posible

def gh(x):
    if x[0]==mayores[v][1]:
        return (x[0], colAct)
    else:
        return x


(grafo_V, grafo_E) = init()

tamMuestraVertices = 20
vColoreados = 0
tope = grafo_V.count()
i = 0

# En grafo_grs, guardo los pares (-gr(v), v)
# Los guardo como -gr(v) para así sacar con takeOrdered los de mayor grado
grafo_grs = grafo_E.map(lambda x: (x[1], x[0]))
grafo_grs = grafo_grs.union(grafo_E).map(lambda x: (x[0], 1)).reduceByKey(lambda a,b: a+b).map(lambda x: (-x[1], x[0]))
#print 'Grados:', grafo_grs.collect()

while(vColoreados < tope):
    mayores = grafo_grs.takeOrdered(tamMuestraVertices)
    #print 'Los mayores:', mayores
    grafo_grs = grafo_grs.subtract(sc.parallelize(mayores)).sortByKey()
    #print 'Los nuevos grados:', grafo_grs.collect()
    print 'Voy por el vertice', vColoreados
    for v in range (0, len(mayores)): # mayores[v][1] indica nodo de mayor grado sin pintar
        pintarV = True
        colAct = 1
        while(pintarV):
            i += 1
            grafo_V = grafo_V.map(gh)
            grafo_V.persist()
            #print grafo_V.collect()
            if(colorVerticeValido(mayores[v][1])):
                pintarV = False
                vColoreados += 1
            else:
                colAct += 1
# Mostrar resultados      
print '-----Resultados-----'
print 'Coloreo final: ', grafo_V.collect()
print 'Iteraciones consumidas: ', i
coloresUsados = grafo_V.map(lambda x: (x[1], x[0])).groupByKey().count()
print 'Cantidad de colores distintos usados: ', coloresUsados


In [ ]:
# Forma de coloreo #3: "Greedy según Matula-Marble-Isaacson"
# 1. Se toma el vértice de menor grado en G y se lo colorea con el menor color posible
# 2. Se hace G = G - v y se vuelve al paso 1 hasta que G esté vacío 

def hoo(x):
    if x[0]==menor[0][1]:
        return (x[0], colAct)
    else:
        return x


(grafo_V, grafo_E) = init()

vColoreados = 0
tope = grafo_V.count()
i = 0
aristas = grafo_E.map(lambda x: x)


while(vColoreados < (tope-1)):
    # En grafo_grs, guardo los pares (gr(v), v)
    grafo_grs = aristas.map(lambda x: (x[1], x[0]))
    grafo_grs = grafo_grs.union(aristas).map(lambda x: (x[0], 1)).reduceByKey(lambda a,b: a+b).map(lambda x: (x[1], x[0]))
    #print 'Grados:', grafo_grs.collect()
    menor = grafo_grs.takeOrdered(1)
    #print 'El nodo menor:   ', menor[0][1]
    #print 'Voy por el vertice', vColoreados
    v = menor[0][1] 
    pintarV = True
    colAct = 1
    print 'Nodo: ', vColoreados
    while(pintarV):
        i += 1
        grafo_V = grafo_V.map(hoo)
        grafo_V.persist()
        #print 'V: ', grafo_V.collect()
        if(colorVerticeValido(v)):
            pintarV = False
            vColoreados += 1
        else:
            colAct += 1
    # Borro el vértice v:
    aristas = aristas.filter(lambda e: ((e[0] != v) and (e[1] != v)))
    #aristas = aristas.subtract(aux)
    #print 'E(G-v): ', aristas.collect()
    #print 'E(G):', grafo_E.collect()
    aristas.persist()
# Mostrar resultados      
print '-----Resultados-----'
print 'Coloreo final: ', grafo_V.collect()
print 'Iteraciones consumidas: ', i
coloresUsados = grafo_V.map(lambda x: (x[1], x[0])).groupByKey().count()
print 'Cantidad de colores distintos usados: ', coloresUsados
    


In [ ]:
# Forma de coloreo #4: "Greedy con cambio de implementación"
# Voy a volver a implementar la forma #2, pero esta vez un grafo será
# un único RDD que represente la lista de adyacencia del mismo. Es decir,
# se guardaran tuplas (vertice, listaDeAdyacencia).
def pintar(x):
    if x[0]==mayores[v][1]:
        return (x[0], colAct)
    else:
        return x
    
def clrVerticeValido(w):
    Vady = (graph.filter(lambda x: x[0] == w).collect())[0]
    for vert in Vady[1]:
        if(colAct == coloresVertices[vert]):
            return False
    return True
    
    
(grafo_V, grafo_E) = init()

graph = grafo_E.flatMap(lambda x: [(x[0], x[1]), (x[1], x[0])]).groupByKey().map(lambda x: (x[0], list(x[1])))
graph = grafo_V.cogroup(graph).map(lambda x: (x[0], list(x[1][1])[0]))
# Tengo ahora en graph las tuplas (vertice, listaDeAdyacencia)
#print graph.collect()

tamMuestraVertices = 60
vColoreados = 0
tope = grafo_V.count()
coloresVertices = []
for w in range (0, tope):
    coloresVertices.append(0)
i = 0

# En grafo_grs, guardo los pares (-gr(v), v)
# Los guardo como -gr(v) para así sacar con takeOrdered los de mayor grado
grafo_grs = grafo_E.map(lambda x: (x[1], x[0]))
grafo_grs = grafo_grs.union(grafo_E).map(lambda x: (x[0], 1)).reduceByKey(lambda a,b: a+b).map(lambda x: (-x[1], x[0]))
#print 'Grados:', grafo_grs.collect()


while(vColoreados < tope):
    mayores = grafo_grs.takeOrdered(tamMuestraVertices)
    #print 'Los mayores:', mayores
    grafo_grs = grafo_grs.subtract(sc.parallelize(mayores))
    #print 'Los nuevos grados:', grafo_grs.collect()
    print 'Voy por el vertice', vColoreados
    for v in range (0, len(mayores)): # mayores[v][1] indica nodo de mayor grado sin pintar
        pintarV = True
        colAct = 1
        while(pintarV):
            i += 1
            #print grafo_V.collect()
            grafo_V = grafo_V.map(pintar)
            grafo_V.persist()
            #print grafo_V.collect()
            if(clrVerticeValido(mayores[v][1])):
                coloresVertices[mayores[v][1]] = colAct
                pintarV = False
                vColoreados += 1
                #print 'Los colores:', coloresVertices
            else:
                colAct += 1
# Mostrar resultados      
print '-----Resultados-----'
#print 'Coloreo final: ', grafo_V.collect()
print 'Coloreo final: ', coloresVertices
print 'Iteraciones consumidas: ', i
#coloresUsados = grafo_V.map(lambda x: (x[1], x[0])).groupByKey().count()
print 'Cantidad de colores distintos usados: ', max(coloresVertices)

